In [8]:
from __future__ import print_function

from keras import backend as K

from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def

from keras.optimizers import SGD
from keras.models import model_from_json
#from keras.models import Model

import shutil
import os
import numpy as np
import tensorflow as tf

from keras.models import load_model
from tensorflow import lite 

In [3]:
json_file = open("purchase_classification_20200205.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into model
loaded_model.load_weights("purchase_classification_20200205.h5")

In [12]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

In [15]:
tf.keras.backend.clear_session()
tf.keras.backend.set_learning_phase(0)

[print(n.name) for n in K.get_session().graph.as_graph_def().node]

frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in loaded_model.outputs])
tf.train.write_graph(frozen_graph, "export", "purchase_classification_20200205.pb", as_text=True)

keras_learning_phase/input
keras_learning_phase


AssertionError: dense_1/Softmax is not in graph